# GVC participation rate

This notebook computes the trade-based and production-based GVC participation rates at the country-level and the 5-, 15-, and 35-sector levels. It uses the trade accounting framework of Borin and Mancini (2019). Results are saved as `gvcp.parquet` in `data/`.

The trade-based GVC participation rate is defined as
```
GVCP_trade_f = (REX1 + REX2 + REX3 + REF1 + REF2) / Exports
GVCP_trade_f = (FVA + PDC1 + PDC2) / Exports
GVCP_trade = GVCP_trade_f + GVCP_trade_b
```
where sectors are broken down by export sectors.

The production-based GVC participation rate, meanwhile, is defined as
```
GVCP_prod = (DAVAX2 + REX1 + REX2 + REX3 + REF1 + REF2) / va
```
where sectors are broken down by origin sectors.

In [9]:
import numpy as np
import pandas as pd
import duckdb

In [10]:
ta = 'ta.parquet'
ta_es = 'ta-es.parquet'
ta_os = 'ta-os.parquet'
summary = 'summary.csv'
outputfile = 'gvcp'

# ta = 'ta62.parquet'
# ta_es = 'ta62-es.parquet'
# ta_os = 'ta62-os.parquet'
# summary = 'summary62.csv'
# outputfile = 'gvcp62'

# ta = 'ta_jun2023.parquet'
# ta_es = 'ta-es_jun2023.parquet'
# ta_os = 'ta-os_jun2023.parquet'
# summary = 'summary_jun2023.csv'
# outputfile = 'gvcp_jun2023'

## Load and process data

### Breakdown by export sectors

In [11]:
df_es = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta_es}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta_es}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta_es}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_es['GVC_trade_f'] = df_es['REX1'] + df_es['REX2'] + df_es['REX3'] + df_es['REF1'] + df_es['REF2']
df_es['GVC_trade_b'] = df_es['FVA'] + df_es['PDC1'] + df_es['PDC2']
df_es['GVC_trade'] = df_es['GVC_trade_f'] + df_es['GVC_trade_b']
df_es['t'] = df_es['t'].astype(int)
df_es = df_es[['t', 's', 'agg', 'i', 'Exports', 'GVC_trade_f', 'GVC_trade_b', 'GVC_trade']]

### Breakdown by origin sectors

In [12]:
df_os = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta_os}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL
    
    (SELECT t, s, 15 AS agg, i15 AS i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta_os}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)
    
    UNION ALL
    
    (SELECT t, s, 35 AS agg, i, 
        SUM(Exports) AS Exports, 
        SUM(DAVAX1) AS DAVAX1,
        SUM(DAVAX2) AS DAVAX2,
        SUM(REX1) AS REX1,
        SUM(REX2) AS REX2,
        SUM(REX3) AS REX3,
        SUM(REF1) AS REF1,
        SUM(REF2) AS REF2,
        SUM(FVA) AS FVA,
        SUM(PDC1) AS PDC1,
        SUM(PDC2) AS PDC2
    FROM read_parquet('../data/trade-accounting/{ta_os}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

df_os['GVC_prod'] = df_os['DAVAX2'] + df_os['REX1'] + df_os['REX2'] + df_os['REX3'] + df_os['REF1'] + df_os['REF2']
df_os['t'] = df_os['t'].astype(int)
df_os = df_os[['t', 's', 'agg', 'i', 'GVC_prod']]

### Value added

In [13]:
va = duckdb.sql(
    f"""
    (SELECT t, s, 0 AS agg, 0 AS i, SUM(va) AS va, 
    FROM read_csv_auto('../data/{summary}')
    GROUP BY t, s
    ORDER BY t, s)

    UNION ALL

    (SELECT t, s, 5 AS agg, i5 AS i, SUM(va) AS va, 
    FROM read_csv_auto('../data/{summary}')
    GROUP BY t, s, i5
    ORDER BY t, s, i5)

    UNION ALL

    (SELECT t, s, 15 AS agg, i15 AS i, SUM(va) AS va, 
    FROM read_csv_auto('../data/{summary}')
    GROUP BY t, s, i15
    ORDER BY t, s, i15)

    UNION ALL

    (SELECT t, s, 35 AS agg, i, SUM(va) AS va, 
    FROM read_csv_auto('../data/{summary}')
    GROUP BY t, s, i
    ORDER BY t, s, i)
    """
).df()

In [14]:
df_os = pd.merge(df_os, va)

## Consolidate and save

In [15]:
df = pd.merge(df_es, df_os)
df['GVCP_trade_f'] = df['GVC_trade_f'] / df['Exports']
df['GVCP_trade_b'] = df['GVC_trade_b'] / df['Exports']
df['GVCP_trade'] = df['GVC_trade'] / df['Exports']
df['GVCP_prod'] = df['GVC_prod'] / df['va']

df = df[[
    't', 's', 'agg', 'i', 'Exports', 'va',
    'GVC_trade_f', 'GVC_trade_b', 'GVC_trade', 'GVC_prod',
    'GVCP_trade_f', 'GVCP_trade_b', 'GVCP_trade', 'GVCP_prod'
]]

df.to_parquet(f'../data/{outputfile}.parquet', index=False)

In [16]:
df

,t,s,agg,i,Exports,va,GVC_trade_f,GVC_trade_b,GVC_trade,GVC_prod,GVCP_trade_f,GVCP_trade_b,GVCP_trade,GVCP_prod
0,2000,1,0,0,91972.342114,385867.994688,22114.076464,13909.627610,36023.704074,59784.722632,0.240443,0.151237,0.391680,0.154936
1,2000,2,0,0,72951.426332,181607.773983,13635.347954,20773.166722,34408.514676,32914.229404,0.186910,0.284753,0.471663,0.181238
2,2000,3,0,0,148839.676022,231768.788150,24199.795197,55287.186218,79486.981415,57323.691415,0.162590,0.371455,0.534044,0.247331
3,2000,4,0,0,1478.645852,12169.272696,311.186573,452.874587,764.061160,785.950320,0.210454,0.306277,0.516730,0.064585
4,2000,5,0,0,64412.402125,621707.487627,12424.996741,7471.122743,19896.119483,36856.009667,0.192898,0.115989,0.308886,0.059282
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56443,2021,63,35,31,19299.751505,576235.047524,4176.187385,2806.787159,6982.974543,15091.292144,0.216386,0.145431,0.361817,0.026189
56444,2021,63,35,32,15712.480730,401361.589117,1764.603303,1905.057183,3669.660486,7906.209734,0.112306,0.121245,0.233551,0.019698
56445,2021,63,35,33,13381.860678,279192.379055,1238.131623,2669.258249,3907.389872,6906.679356,0.092523,0.199468,0.291992,0.024738
56446,2021,63,35,34,19335.567305,207665.098239,3224.894415,3754.103151,6978.997566,21461.015814,0.166786,0.194155,0.360941,0.103344
